In [1]:
import simdjson as json
import copy
import os 

import numpy as np
from process_bigraph import pp
from vivarium.core.composer import Composer

from ecoli.experiments.ecoli_master_sim import EcoliSim
from ecoli.shared.registry import ecoli_core


def format_config(d: dict, instance=None):
    import numpy as np
    import unum
    """Returns a dict whose keys match that of d, except replacing innermost values (v) with their corresponding _default declarations.
    Used for migration.
    """
    if isinstance(d, dict):
        for key, val in d.items():
            if isinstance(val, np.ndarray): 
                d[key] = val.tolist()
            if isinstance(val, set):
                d[key] = list(val)
            if isinstance(val, unum.Unum):
                d[key] = str(val)
            if type(val).__name__ == "function":
                if instance is not None:
                    setattr(instance, key, val)
                d[key] = key
        else:
            return {k: format_config(v, instance) for k, v in d.items()}
    else:
        return d 
    

def extract_process_states(write: bool = False):
    sim = EcoliSim.from_file()
    sim.build_ecoli()
    processes = sim.processes

    data_dir = '/Users/alexanderpatrie/Desktop/repos/ecoli/v2Ecoli/data/model'
    topology_fp = os.path.join(data_dir, 'single_topology.json')
    with open(topology_fp, 'r') as f:
        raw_state = json.load(f)
    
    state = {}
    for process_id, process_ports in raw_state.items():
        process_spec = {
            "address": f"local:{process_id}",
            "inputs": process_ports,
            "outputs": process_ports
        }

        # attempt config extract if instance is available
        if process_id in processes:
            # TODO: this needs to be serialized for arrays
            instance = processes[process_id]
            process_config = copy.deepcopy(instance.defaults)
            process_config = format_config(
                process_config, 
                instance
            )
            process_spec["config"] = process_config

        state[process_id] = process_spec
    
    # TODO: export doc here
    if write:
        state_fp = os.path.join(data_dir, 'single_state.json')
        with open(state_fp, 'w') as f:
            json.dump(state, f, indent=4)
    return state

state = extract_process_states(write=True)
state

<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


{'RNA_counts_listener': {'address': 'local:RNA_counts_listener',
  'inputs': {'listeners': ['listeners'],
   'RNAs': ['unique', 'RNA'],
   'global_time': ['global_time'],
   'timestep': ['timestep']},
  'outputs': {'listeners': ['listeners'],
   'RNAs': ['unique', 'RNA'],
   'global_time': ['global_time'],
   'timestep': ['timestep']},
  'config': {'rna_ids': [],
   'mrna_indexes': [],
   'time_step': 1,
   'emit_unique': False}},
 'bulk-timeline': {'address': 'local:bulk-timeline',
  'inputs': {'bulk': ['bulk'],
   'global': ['timeline'],
   'media_id': ['environment', 'media_id']},
  'outputs': {'bulk': ['bulk'],
   'global': ['timeline'],
   'media_id': ['environment', 'media_id']},
  'config': {'time_step': 1.0, 'timeline': []}},
 'dna_supercoiling_listener': {'address': 'local:dna_supercoiling_listener',
  'inputs': {'listeners': ['listeners'],
   'chromosomal_segments': ['unique', 'chromosomal_segment'],
   'global_time': ['global_time'],
   'timestep': ['timestep']},
  'outputs'

In [2]:
x = {}
if not x:
    print(True)

True


In [24]:
i = dict(zip(
    ['x', 'y', 'z'], 
    [{'value': np.random.random((11,))} for _ in range(3)]
))
j = dict(zip(
    ['x', 'y', 'z'], 
    [{'value': np.random.random((11,))} for _ in range(3)]
))
k = {**j, 'make_rates': lambda v: [i**i for i in range(v)]}
i.update(k)
v = copy.deepcopy(i)
nesting = 11
x = dict(zip(
    [str(np.random.random()) for _ in range(nesting)],
    [{str(k): v for k in range(nesting)}]
))

In [42]:
from functools import partial 
import numpy as np 


make_rates = lambda: np.random.random()**np.random.random() / np.pi
conf = {'x': make_rates, 'y': np.random.random((11,))}
format_config(conf)

'x'

'x'

In [36]:
conf

{'x': <function __main__.<lambda>()>,
 'y': array([0.2720273 , 0.93367277, 0.97837976, 0.59859793, 0.42651113,
        0.49828625, 0.4229258 , 0.74594848, 0.28782849, 0.80921659,
        0.17371964])}

In [39]:
def f(x: float):
    pass 

func = lambda: []

type(func)

function

In [8]:
from vivarium import Vivarium 

v = Vivarium(core=ecoli_core, processes=ecoli_core.process_registry.registry)

v.add_process('grow', 'grow', inputs={'mass': ['mass']}, outputs={'mass': ['mass']})
doc = v.make_document()
doc['state']['grow'].pop('config')

{'rate': '0.0'}

In [11]:
doc
v2 = Vivarium(core=ecoli_core, processes=ecoli_core.process_registry.registry, document=doc)

In [12]:
v2

Vivarium( 
{ 'global_time': 0.0,
  'grow': { 'address': 'local:grow',
            'config': 'default_quote',
            'inputs': {'mass': ['mass']},
            'instance': <process_bigraph.processes.growth_division.Grow object at 0x17feba030>,
            'interval': 1.0,
            'outputs': {'mass': ['mass']},
            'shared': {}},
  'mass': 0.0})

In [13]:
v2.add_emitter()
v2.run(11)
v2.get_results()

TypeError: string indices must be integers, not 'str'

In [41]:
import numpy as np 


def collapse_defaults(d):
    """Returns a dict whose keys match that of d, except replacing innermost values (v) with their corresponding _default declarations.
    Used for migration.
    """
    if isinstance(d, dict):
        for key, val in d.items():
            if isinstance(val, np.ndarray):
                return val.tolist()
        else:
            return {k: collapse_defaults(v) for k, v in d.items()}
    else:
        return d


i = dict(zip(
    ['x', 'y', 'z'], 
    [{'value': np.random.random((11,))} for _ in range(3)]
))
j = dict(zip(
    ['x', 'y', 'z'], 
    [{'value': np.random.random((11,))} for _ in range(3)]
))
i.update(j)
v = copy.deepcopy(i)
nesting = 11
x = dict(zip(
    [str(np.random.random()) for _ in range(nesting)],
    [{str(k): v for k in range(nesting)}]
))
format_config(x)



{'0.5676390918844227': {'0': {'x': [0.12596087681114132,
    0.5981295953929078,
    0.32667914321393876,
    0.0949022571087339,
    0.5427932348144535,
    0.21022633822617687,
    0.10275601063080642,
    0.026909889611403504,
    0.5175723248131481,
    0.1261947970643612,
    0.013061408322747159],
   'y': [0.36058978174811596,
    0.42936463541792025,
    0.704847640045133,
    0.7122340395419946,
    0.4344179787660949,
    0.9700636979331491,
    0.2880227802812595,
    0.05451306621599061,
    0.028729910594651686,
    0.314618367231007,
    0.12388164356677522],
   'z': [0.4433757828079852,
    0.8455308974411397,
    0.29349477941740676,
    0.1786299716455405,
    0.8784472566695652,
    0.9787144497937318,
    0.1822433961802643,
    0.8765926810346769,
    0.047848851563784045,
    0.5784572698796869,
    0.026453110316221773]},
  '1': {'x': [0.12596087681114132,
    0.5981295953929078,
    0.32667914321393876,
    0.0949022571087339,
    0.5427932348144535,
    0.2102263